# Portfolio Differential Evolution

In [142]:
import os
import time 
  

In [143]:
def WeightStdzn130_30BoundsConstr(InpWeightMat, LowUpBounds):
    
    #dependencies
    import numpy as np
    
    WeightMatrix = InpWeightMat
    [RowsMat, ColsMat]=np.shape(WeightMatrix) 
    [LowBound, UpBound] = np.shape(LowUpBounds) 
    
        
    # Steps 1 and 2 of Weight Repair Strategy Phase 1
    # Standardize weights to satisfy their lower bounds while  ensuring
    # that their sum equals 1
    for i in range(RowsMat):
        #R: those weights which are less than their respective lower bounds
        R = [] 
        for j in range(ColsMat):
            
            if (WeightMatrix[i,j]< LowUpBounds[0,j]):
                WeightMatrix[i,j]= LowUpBounds[0,j] 
                R.append(j)
                
        #Q: those weights which satisfy their lower bounds
        Q = list(set(range(ColsMat))-set(R))
        F = 1.0 - np.sum(LowUpBounds[0,R])- np.sum(LowUpBounds[0,Q]) 
        L = np.sum(np.abs(WeightMatrix[i,Q])) 
        if (L==0):
            term = F / len(Q) 
            WeightMatrix[i,Q]= LowUpBounds[0,Q]+ term 
        else:
            term = F / L 
            WeightMatrix[i,Q] = LowUpBounds[0,Q]+ np.abs(WeightMatrix[i,Q])* term 
            
        
    # Steps 3-6 of Weight Repair Strategy Phase 1 
    # standardize upper bounds so that weights ultimately satisfy both upper
    # and lower bounds while their sum equals 1
    for i in range(RowsMat): 
        
        r = [] 
                    
        ExitFlag =  True 
        q = list(set(range(ColsMat))-set(r))
        
        while (ExitFlag ==  True): 
            ExitFlag =  False  
           
            for j in range(len(q)):
                if ( WeightMatrix[i, q[j]] <= LowUpBounds[1, q[j]] ):
                    continue 
                else:
                    ExitFlag =  True 
                    r.append(q[j]) 
                    
            q = list(set(range(ColsMat))-set(r))
            if ( ExitFlag ==  True):
                L =np.sum(np.abs(WeightMatrix[i,q])) 
                F = 1.0 - ( np.sum(LowUpBounds[0,q])+ np.sum( LowUpBounds[1,r]) ) 
                if (L==0): 
                    term = F 
                    WeightMatrix[i,q[0]]= term 
                else:
                    term = F/L 
                    WeightMatrix[i,q] = LowUpBounds[0,q] + (np.abs(WeightMatrix[i,q])* term) 
                    
                WeightMatrix[i,r]=LowUpBounds[1,r] 
                
    StdWeightMatrix = WeightMatrix
    
    
    return StdWeightMatrix

In [144]:
def WeightStdzn130_30BudgetConstr(InputWeightMatrix, LongLowUpBounds, ShortLowUpBounds):
    
    #dependencies
    import numpy as np
    
    WeightMatrix = InputWeightMatrix
    
    [RowsMat, ColsMat]= np.shape(WeightMatrix)
    StdzWeightMatrix = np.zeros([RowsMat, ColsMat], dtype = float)
    
    #  budgets for long positions
    eta = 1.3  
    gamma = 0  
    for p in range(RowsMat):
        
        #  identify long and short positions in H, where H[0] denotes
        #  indices of long positions and H[1] denotes indices of short positions
        H = GroupAssets130_30(ColsMat, WeightMatrix[p,:]) 
        
        
        #  Adjust weights representing long positions
        DepositWeights = 0.0  
        h = 0  
        SumWeights = np.sum(WeightMatrix[p,H[h]])

        if ((SumWeights <= eta ) & (SumWeights >= gamma)):
            continue  
        else:                       
            DepositWeights = DepositWeights + (SumWeights-eta)  
            MP = eta  
            WeightMatrix[p,:] = StdznAssetWeights130_30LongPositions(WeightMatrix[p,:], H[h], SumWeights, MP, LongLowUpBounds)  
           
        #  Adjust weights representing short positions
        if (DepositWeights ==0):
            continue  
        else:
            h = 1   
            SumWeights = np.sum(WeightMatrix[p,H[h]])  
            AbsSumWeights = np.sum(np.abs(WeightMatrix[p,H[h]]) )  
            DepositWeights = DepositWeights + SumWeights  
            WeightMatrix[p,:] = StdznAssetWeights130_30ShortPositions(WeightMatrix[p,:], H[h], AbsSumWeights, DepositWeights, ShortLowUpBounds)  
           
    StdzWeightMatrix = WeightMatrix
    return StdzWeightMatrix

In [145]:
def GroupAssets130_30(PortfolioSize, chromosome):
    
    #dependencies
    import numpy as np
    
    #initialization
    LongPositions = []
    ShortPositions = []
    
    for c in range(PortfolioSize):
        if (chromosome[c] >= 0):
            LongPositions.append(c)
        else:
            ShortPositions.append(c)
            
    GroupAssets = [LongPositions, ShortPositions]
     
    return GroupAssets

In [146]:
def  StdznAssetWeights130_30LongPositions(InputWeightVector, M, WeightSum, TW, LowUpBounds):
    
    #dependencies
    import numpy as np
       
    WeightVector = InputWeightVector 
                         
    # adjust lower bounds of assets                        
    L = WeightSum   
    F =  TW - np.sum(LowUpBounds[0,M])
    term = F /L 
    WeightVector[M] = LowUpBounds[0,M]+ (WeightVector[M]* term) 

                     
    # adjust upper bounds of assets 
    r = [] 
    ExitFlag =  True 
    q = M 
    while (ExitFlag ==  True) :
        ExitFlag = False  
        for j in range(len(q)):
            if (WeightVector[q[j]]> LowUpBounds[1,q[j]]):
                ExitFlag =  True 
                r.append(q[j]) 
         
        q = list(set(M)-set(r))
        if (ExitFlag ==  True):
            L = np.sum(WeightVector[q]) 
            F = TW - (np.sum(LowUpBounds[0,q])+ np.sum( LowUpBounds[1, r]) ) 
            term = F/L 
            WeightVector[q] = LowUpBounds[0,q] + (WeightVector[q]* term) 
            WeightVector[r] = LowUpBounds[1,r]

             
    return WeightVector

In [147]:
def StdznAssetWeights130_30ShortPositions(InpWeightVector, M, AbsWeightSum, TW, LowUpBounds):       
      
    #dependencies 
    import numpy as np 
         
    WeightVector = InpWeightVector
      
    # adjust lower bounds of assets, where TW is the budget imposed on
    # short positions
    L = AbsWeightSum   
    F =  TW - np.sum(LowUpBounds[0,M])  
    if (L==0):
        term = F/len(M) 
        WeightVector[M] = LowUpBounds[0,M]+ term 
    else:
        term = F /L 
        WeightVector[M] = LowUpBounds[0, M]+ (np.abs(WeightVector[M])* term)          
      
    # adjust upper bounds of assets 
    r = [] 
    ExitFlag = True 
    q = M
    while (ExitFlag == True):
        ExitFlag = False  
        for j in range(len(q)):
            if (WeightVector[q[j]] > LowUpBounds[1,q[j]]):
                ExitFlag = True 
                r.append (q[j])
         
        q = list(set(M)-set(r))  
        if (ExitFlag == True):
            L = np.sum(np.abs(WeightVector[q])) 
            F = TW - (np.sum(LowUpBounds[0,q])+ np.sum(LowUpBounds[1,r]) ) 
            if(len(q)!=0):
                if (L==0):
                    term = F 
                    WeightVector[q[0]]= term 
                else:
                    term = F/L 
                    WeightVector[q] = LowUpBounds[0,q] + (np.abs(WeightVector[q])* term) 
            WeightVector[r] = LowUpBounds[1, r] 
         
    return WeightVector

In [148]:
def  ConstrViolnFunction130_30(WeightMat, BetasAssets, C_param, AlphaParam, beta_param, GenerationCount):
    
    #dependencies
    import numpy as np
    
    [RowMat, ColMat]= np.shape(WeightMat)
    epsilon = 0.001
    
    G = np.zeros(shape =(RowMat))
    psi = np.zeros(shape = (RowMat))
    for i in range(RowMat):  
        
        ChromosomeX = WeightMat[i,:]
        
        # compute penalty function G
        PortfolioBetaTerm = np.sum(np.multiply(BetasAssets, ChromosomeX)) 
        g1Term = np.abs((PortfolioBetaTerm) - 1.0)-epsilon
        
        if (g1Term <=0 ):
            G[i]=0
        else:
            G[i]=1
            
        #compute constraint violation function
        PenaltyTerm= np.power((C_param * GenerationCount), AlphaParam) 
        psi[i] = PenaltyTerm *( G[i] * np.power(g1Term, beta_param))
            
    return [psi, G]

In [149]:
def ComputeFitness130_30(PoplnMat, ReturnData,  CovarianceData, RiskFreeData,  PsiFunction):
    
    #dependencies
    import numpy as np
    
    [popln_size, cols]= np.shape(PoplnMat)
    weight = np.zeros(cols)
    
    PoplnFitness = np.zeros(popln_size)
    
    for i in range(popln_size):
        weight = PoplnMat[i,:] 
        PoplnFitness[i] = ((np.matmul(ReturnData,  weight.T)-RiskFreeData)/np.sqrt(np.matmul( np.matmul(weight, CovarianceData), weight.T)) ) - PsiFunction[i]  
    
    return PoplnFitness

In [150]:
def  DEOperatorRand4BestDir5(popln, FitnessValue,  BetaValue, PoplnSize, IndividualLength):
    
    #dependencies
    import numpy as np
    import array as arr
    
    #initialization
    TrialVectorPopln  = np.zeros([PoplnSize, IndividualLength])
    
    for i in range(PoplnSize):
        DifferentialVecIndex = [0]*5
        V  = np.zeros([4,IndividualLength])
        Vb = np.zeros(IndividualLength)
     
        #set IND the current individual in the population indicated by i
        IND = popln[i,:]
        
     
       # prepare RandomIndex, random number indices for each population
       # individual to enable it choose five random individuals from the
       # population, without repeating itself.
        RandomIndex = arr.array('i', np.random.permutation(PoplnSize))
    
        RandomIndex.remove(i)
        
       # select five random individuals from the population
     
        for u in range (5):
            DifferentialVecIndex[u] = int(RandomIndex[u]) 
         
        
        # Obtain Vb the best individual with the 
        # maximal objective function value and designate the rest as array V
        individuals = [FitnessValue[DifferentialVecIndex[0]], FitnessValue[DifferentialVecIndex[1]], FitnessValue[DifferentialVecIndex[2]], FitnessValue[DifferentialVecIndex[3]], FitnessValue[DifferentialVecIndex[4]]]
        
        max_obj_indx = individuals.index (np.max(individuals))  
        j = 0  
        for z in range(5):
            if (DifferentialVecIndex[z] == DifferentialVecIndex[max_obj_indx] ): 
                Vb = popln[DifferentialVecIndex[z], : ] 
                 
            else:
                V[j,:] = popln[DifferentialVecIndex[z],: ]  
                j=j+1  
        
 
        # obtain trial vector for each of the parent vector individual
        TrialVectorPopln[i,:] = Vb + BetaValue/5 * ( 5*Vb - IND - V[0,:]-V[1,:]-V[2,:]-V[3,:])  
        
    return TrialVectorPopln

In [151]:
def  DEOperatorBinCrossover (ParentPopln, TargetVecPopln, ProbabilityRecombn, Components):
    
    #dependencies
    import numpy as np
    
    [row, col] = np.shape(ParentPopln)
    tau = DEComputeTau (Components, ProbabilityRecombn)
    
    OffspringPopln = np.empty(shape =(row, col), dtype = float)
    for i in range(col):
        if i in tau:
            OffspringPopln[:, i]= TargetVecPopln[:,i]
        else: 
            OffspringPopln[:, i] = ParentPopln[:,i]
            
    return OffspringPopln

In [152]:
def DEComputeTau(GeneSize, ProbabilityRecombn):
    
    #dependencies
    import numpy as np
    import random
    
    h = list(np.random.permutation(range(GeneSize)))
    
    #set jStar to a random index
    jStar = h[0]   
    
    tau=[jStar] 
    
    h.remove(jStar)
    
    for i in range(GeneSize-1):
        if (random.random()< ProbabilityRecombn):
            tau.append(h[i])
            
    return tau

In [153]:
def DEOperatorDetermSelection(FeasParentPopln, FeasParentPoplnFitness,PsiParent, FeasOffsprngPopln,FeasOffsprngPoplnFitness, PsiOffsprng,  PoplnSize): 
    
    #dependencies
    import numpy as np
    
    [row, col] = np.shape(FeasParentPopln)
    
    #initialization
    NextGenPool = np.zeros(shape = (PoplnSize, col), dtype = float)
    NextGenPoolFitness = np.zeros(shape = (PoplnSize), dtype = float)
    NextGenPoolPsi = np.zeros(shape = (PoplnSize), dtype = float)
    
    for i in range (PoplnSize):
        if (FeasParentPoplnFitness[i] >= FeasOffsprngPoplnFitness[i]):
            NextGenPool[i,:]= FeasParentPopln[i,:] 
            NextGenPoolPsi[i]= PsiParent[i] 
            NextGenPoolFitness[i] = FeasParentPoplnFitness[i] 
        else: 
            NextGenPool[i,:]= FeasOffsprngPopln [i,:] 
            NextGenPoolPsi[i]=PsiOffsprng[i] 
            NextGenPoolFitness[i] = FeasOffsprngPoplnFitness[i] 
    
    return [NextGenPool, NextGenPoolFitness, NextGenPoolPsi]

In [154]:
data = pd.read_csv('./Data_De/2013-01-01-2014-12-31.csv')
data.shape[1]

465

In [156]:
#dependencies
import numpy as np
import pandas as pd
import array as arr
import csv
import random


# Read csv file to obtain asset labels, mean returns, 
# variance-covariance matrix of asset returns and asset betas
portfolioSize = data.shape[1]
rows = data.shape[0]

stockParamsFileName = './Data_De/2013-01-01-2014-12-31.csv'
df = pd.read_csv(stockParamsFileName,  nrows= rows)

# extract asset labels
assetLabels = df.columns.tolist()[0:portfolioSize]
print(assetLabels)

# extract mean returns, variance-covariance matrix of returns and asset betas
stockParamsData = np.array(df.iloc[0:, 0:]) 
meanData = np.array(stockParamsData[0,:])
covData = np.array(stockParamsData[1:portfolioSize+1, :])
betaAssets = np.array(stockParamsData[portfolioSize+1,:]) 

# general bounds
bounds = np.vstack( (np.repeat(-0.3, portfolioSize), np.repeat(1.3, portfolioSize)))

# set bounds for the long and short positions in the portfolio as (0, 1.3) and (-0.3, 0) respectively
longPosBounds = np.vstack( (np.repeat(0, portfolioSize), np.repeat(1.3, portfolioSize)))
shortPosBounds = np.vstack( (np.repeat(-0.3, portfolioSize), np.repeat(0, portfolioSize)))

# Risk free rate 
annRiskFree = 6.5/100 
riskfree = (np.power((1+annRiskFree),(1.0/360)) -1.0)*100 
  
# Differential Evolution strategy parameters
poplnSize = 400  
chromosomeLength = portfolioSize  
totalGenerations = 50  
beta = 0.5 
probabilityRecombination = 0.87  

# Joines and Houck's (1994)  dynamic penalty (dp) function strategy
# Set constants C, alpha, beta
CConstant = 0.5  
alphaConstant=2 
betaConstant=2 

# initialize Hall of Fame which will finally hold the optimal weights       
HOFFitness = -999.0 
HOFIndividual = np.zeros(shape=(portfolioSize), dtype = float) 
i1 = 0 
 
# generation counter
generationCount = 1  

# generate initial random population within the range [-0.3, 1.3]
initialPoplnSeed =  np.random.uniform(low = -0.3, high = 1.3, size =(poplnSize, chromosomeLength)) 

# repair weights
initialPoplnBound = WeightStdzn130_30BoundsConstr(initialPoplnSeed, bounds) 
initialPopln = WeightStdzn130_30BudgetConstr(initialPoplnBound, longPosBounds, shortPosBounds) 
# compute constraint violation function
[initialPoplnPsi, initialPoplnG1]= ConstrViolnFunction130_30(initialPopln, betaAssets, CConstant, alphaConstant,
                                                             betaConstant, generationCount) 
# compute fitness function
initialPoplnFitness = ComputeFitness130_30(initialPopln, meanData, covData, riskfree,  initialPoplnPsi) 
                                      
       
# create parent population and compute fitness       
feasParentPopln = initialPopln  
feasParentPoplnFitness = initialPoplnFitness 
feasParentPoplnPsi = initialPoplnPsi 

#initialize HOF
HOFGenerationArray = np.zeros(poplnSize)
HOFFitnessArray = np.zeros(poplnSize)
# while loop for generation cycles begins      
        
while (generationCount <= totalGenerations):         
        print('generation:',generationCount)                                      
        dynamicBeta = 0.0
        # dynamic beta for each generation
        dynamicBeta = np.random.uniform(low =0.5, high =1.0)
        
        
        # obtain trial vector population
        trialVectorPopln = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)  
        trialVectorPopln = DEOperatorRand4BestDir5(feasParentPopln, feasParentPoplnFitness,  dynamicBeta, poplnSize, chromosomeLength) 
        
        # obtain offspring population
        offspringPopln = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
        offspringPopln = DEOperatorBinCrossover(feasParentPopln, trialVectorPopln, probabilityRecombination, chromosomeLength) 
          
        # undertake weight repair strategy Phase 1
        mutatedPoplnBound = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
        mutatedPoplnBound = WeightStdzn130_30BoundsConstr(offspringPopln, bounds) 
        
        # undertake weight repair strategy Phase 2
        feasMutatedPopln = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
        feasMutatedPopln = WeightStdzn130_30BudgetConstr(mutatedPoplnBound, longPosBounds, shortPosBounds) 
        
        # compute constraint violation function
        feasMutatedPoplnPsi = np.zeros(shape =(poplnSize), dtype = float)
        feasMutatedPopln_G1 = np.zeros(shape =(poplnSize), dtype = float)
          
        [feasMutatedPoplnPsi, feasMutatedPopln_G1]= ConstrViolnFunction130_30(feasMutatedPopln, betaAssets, CConstant, alphaConstant, betaConstant, generationCount) 
        
          
        # compute fitness function values
        feasMutatedPoplnFitness = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
        feasMutatedPoplnFitness = ComputeFitness130_30(feasMutatedPopln, meanData, covData, riskfree,  feasMutatedPoplnPsi) 
        
        # set the population for the next generation
        nextGenPool = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
        nextGenPoolFitness = np.zeros(shape =(poplnSize), dtype = float)
        psiFun= np.zeros(shape =(poplnSize), dtype = float)
         
          
        [nextGenPool, nextGenPoolFitness, psiFun ] = DEOperatorDetermSelection(feasParentPopln, feasParentPoplnFitness,  feasParentPoplnPsi,   feasMutatedPopln, feasMutatedPoplnFitness, feasMutatedPoplnPsi,  poplnSize)  
        
        
        # induct best individual into Hall of Fame
        for i in range(poplnSize):
            if (psiFun[i] == 0):
                                       
                    if (nextGenPoolFitness[i] > HOFFitness):
                        HOFFitness = nextGenPoolFitness[i]
                        HOFIndividual = nextGenPool[i,:] 
                        HOFGenerationArray[i1] = generationCount 
                        HOFFitnessArray[i1] = HOFFitness 
                        print('HOF updated Generation', generationCount)
                        print('HOF fitness', HOFFitness)
                        i1=i1+1 
                
        
        # increment generation counter
        generationCount = generationCount + 1
        feasParentPopln = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
        feasParentPoplnFitness = np.zeros(shape =(poplnSize), dtype = float)
        
        # set the parent population for the next generation
        feasParentPopln = nextGenPool 
        feasParentPoplnFitness = nextGenPoolFitness 
        
# while loop for DE HOF generations ends
        

# obtain optimal weights from the Hall of Fame 
xStar = HOFIndividual  
    
#   compute optimal portfolio annualized risk and return            
portfolioReturn =  1 * np.sum(np.multiply(meanData, xStar)) 
risk = np.sqrt(1 * np.matmul( np.matmul(xStar, covData), xStar.T)) 
   
print("Integrated 130-30 Portfolio Optimization results:")
print("Annualized risk ( %) ", risk, " Expected Portfolio Annualized return ( %)",  portfolioReturn)

sharpeRatio = (portfolioReturn-annRiskFree*100.0)/ risk
print("Sharpe Ratio", sharpeRatio)

    
longIndex = np.where(xStar > 0)
shortIndex = np.where(xStar < 0)
print("Long positions of the optimal 130-30 portfolio: ", np.array(assetLabels)[longIndex])
print("Short positions of the optimal 130-30 portfolio: ", np.array(assetLabels)[shortIndex])

print("Optimal Weights: long positions: ", xStar[longIndex]) 
print("Optimal Weights: short positions: ", xStar[shortIndex])


print("Sum of long position weights: ",  np.sum(xStar[longIndex]))
print("Sum of short position weights:  ", np.sum(xStar[shortIndex]))
print("Sum of weights: " ,      np.sum(xStar))

portfoliobeta = np.sum(np.multiply(xStar, betaAssets))
print("Portfolio Beta: ", portfoliobeta)

    
print('Successful execution!')

['A', 'AAL', 'AAP', 'AAPL', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE', 'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA', 'BAC', 'BAX', 'BBY', 'BDX', 'BEN', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BR', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CBOE', 'CBRE', 'CCI', 'CCL', 'CDNS', 'CE', 'CERN', 'CF', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF', 'COG', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'CTXS', 'CVS', 'CVX', 'CXO', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISCK', 'DISH', 'DLR', 'DLTR', 'DOV', 'DPZ', 'DRE', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXC', 'DXCM', 'EA

In [157]:
xStar.shape

(465,)

In [158]:
#assetLabels

In [159]:
salida = pd.DataFrame(list(zip(assetLabels,xStar))).transpose()
salida.columns = salida.iloc[0]
salida.drop([0], inplace=True)

In [160]:
salida

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION
1,0,0.0363165,0,0,0.0363165,0,0.0363165,0,0,0,...,0,0,0.0363165,0,0,0,0.00545516,0,0,0


##  para todos

In [161]:
def portfolio_DE(path_input, n_generations, rf):
    data = pd.read_csv(path_input)
    # Read csv file to obtain asset labels, mean returns, 
    # variance-covariance matrix of asset returns and asset betas
    portfolioSize = data.shape[1]
    rows = data.shape[0]

    stockParamsFileName = path_input
    df = pd.read_csv(stockParamsFileName,  nrows= rows)

    # extract asset labels
    assetLabels = df.columns.tolist()[0:portfolioSize]
    #print(assetLabels)

    # extract mean returns, variance-covariance matrix of returns and asset betas
    stockParamsData = np.array(df.iloc[0:, 0:]) 
    meanData = np.array(stockParamsData[0,:])
    covData = np.array(stockParamsData[1:portfolioSize+1, :])
    betaAssets = np.array(stockParamsData[portfolioSize+1,:]) 

    # general bounds
    bounds = np.vstack( (np.repeat(-0.3, portfolioSize), np.repeat(1.3, portfolioSize)))

    # set bounds for the long and short positions in the portfolio as (0, 1.3) and (-0.3, 0) respectively
    longPosBounds = np.vstack( (np.repeat(0, portfolioSize), np.repeat(1.3, portfolioSize)))
    shortPosBounds = np.vstack( (np.repeat(-0.3, portfolioSize), np.repeat(0, portfolioSize)))

    # Risk free rate 
    annRiskFree = 6.5/100 
    riskfree = (np.power((1+annRiskFree),(1.0/360)) -1.0)*100 

    # Differential Evolution strategy parameters
    poplnSize = 400  
    chromosomeLength = portfolioSize  
    totalGenerations = n_generations  
    beta = 0.5 
    probabilityRecombination = 0.87  

    # Joines and Houck's (1994)  dynamic penalty (dp) function strategy
    # Set constants C, alpha, beta
    CConstant = 0.5  
    alphaConstant=2 
    betaConstant=2 

    # initialize Hall of Fame which will finally hold the optimal weights       
    HOFFitness = -999.0 
    HOFIndividual = np.zeros(shape=(portfolioSize), dtype = float) 
    i1 = 0 

    # generation counter
    generationCount = 1  

    # generate initial random population within the range [-0.3, 1.3]
    initialPoplnSeed =  np.random.uniform(low = -0.3, high = 1.3, size =(poplnSize, chromosomeLength)) 

    # repair weights
    initialPoplnBound = WeightStdzn130_30BoundsConstr(initialPoplnSeed, bounds) 
    initialPopln = WeightStdzn130_30BudgetConstr(initialPoplnBound, longPosBounds, shortPosBounds) 
    # compute constraint violation function
    [initialPoplnPsi, initialPoplnG1]= ConstrViolnFunction130_30(initialPopln, betaAssets, CConstant, alphaConstant,
                                                                 betaConstant, generationCount) 
    # compute fitness function
    initialPoplnFitness = ComputeFitness130_30(initialPopln, meanData, covData, riskfree,  initialPoplnPsi) 


    # create parent population and compute fitness       
    feasParentPopln = initialPopln  
    feasParentPoplnFitness = initialPoplnFitness 
    feasParentPoplnPsi = initialPoplnPsi 

    #initialize HOF
    HOFGenerationArray = np.zeros(poplnSize)
    HOFFitnessArray = np.zeros(poplnSize)
    # while loop for generation cycles begins      

    while (generationCount <= totalGenerations):         
            print('generation:',generationCount)                                      
            dynamicBeta = 0.0
            # dynamic beta for each generation
            dynamicBeta = np.random.uniform(low =0.5, high =1.0)


            # obtain trial vector population
            trialVectorPopln = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)  
            trialVectorPopln = DEOperatorRand4BestDir5(feasParentPopln, feasParentPoplnFitness,  dynamicBeta, poplnSize, chromosomeLength) 

            # obtain offspring population
            offspringPopln = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
            offspringPopln = DEOperatorBinCrossover(feasParentPopln, trialVectorPopln, probabilityRecombination, chromosomeLength) 

            # undertake weight repair strategy Phase 1
            mutatedPoplnBound = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
            mutatedPoplnBound = WeightStdzn130_30BoundsConstr(offspringPopln, bounds) 

            # undertake weight repair strategy Phase 2
            feasMutatedPopln = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
            feasMutatedPopln = WeightStdzn130_30BudgetConstr(mutatedPoplnBound, longPosBounds, shortPosBounds) 

            # compute constraint violation function
            feasMutatedPoplnPsi = np.zeros(shape =(poplnSize), dtype = float)
            feasMutatedPopln_G1 = np.zeros(shape =(poplnSize), dtype = float)

            [feasMutatedPoplnPsi, feasMutatedPopln_G1]= ConstrViolnFunction130_30(feasMutatedPopln, betaAssets, CConstant, alphaConstant, betaConstant, generationCount) 


            # compute fitness function values
            feasMutatedPoplnFitness = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
            feasMutatedPoplnFitness = ComputeFitness130_30(feasMutatedPopln, meanData, covData, riskfree,  feasMutatedPoplnPsi) 

            # set the population for the next generation
            nextGenPool = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
            nextGenPoolFitness = np.zeros(shape =(poplnSize), dtype = float)
            psiFun= np.zeros(shape =(poplnSize), dtype = float)


            [nextGenPool, nextGenPoolFitness, psiFun ] = DEOperatorDetermSelection(feasParentPopln, feasParentPoplnFitness,  feasParentPoplnPsi,   feasMutatedPopln, feasMutatedPoplnFitness, feasMutatedPoplnPsi,  poplnSize)  


            # induct best individual into Hall of Fame
            for i in range(poplnSize):
                if (psiFun[i] == 0):

                        if (nextGenPoolFitness[i] > HOFFitness):
                            HOFFitness = nextGenPoolFitness[i]
                            HOFIndividual = nextGenPool[i,:] 
                            HOFGenerationArray[i1] = generationCount 
                            HOFFitnessArray[i1] = HOFFitness 
                            print('HOF updated Generation', generationCount)
                            print('HOF fitness', HOFFitness)
                            i1=i1+1 


            # increment generation counter
            generationCount = generationCount + 1
            feasParentPopln = np.zeros(shape =(poplnSize, chromosomeLength), dtype = float)
            feasParentPoplnFitness = np.zeros(shape =(poplnSize), dtype = float)

            # set the parent population for the next generation
            feasParentPopln = nextGenPool 
            feasParentPoplnFitness = nextGenPoolFitness 

    # while loop for DE HOF generations ends


    # obtain optimal weights from the Hall of Fame 
    xStar = HOFIndividual  

    #   compute optimal portfolio annualized risk and return            
    portfolioReturn =  1 * np.sum(np.multiply(meanData, xStar)) # los retornos estan anualizados
    risk = np.sqrt(253 * np.matmul( np.matmul(xStar, covData), xStar.T))  #se debe anualziar la volatilidad

    #print("Integrated 130-30 Portfolio Optimization results:")
    #print("Annualized risk ( %) ", risk, " Expected Portfolio Annualized return ( %)",  portfolioReturn)
    print ("retorno:", portfolioReturn)
    print ("risk:", risk)
    print ("rf:", rf)
    sharpeRatio = (portfolioReturn-rf)/ risk
    print("Sharpe Ratio", sharpeRatio)


    longIndex = np.where(xStar > 0)
    shortIndex = np.where(xStar < 0)
    #print("Long positions of the optimal 130-30 portfolio: ", np.array(assetLabels)[longIndex])
    #print("Short positions of the optimal 130-30 portfolio: ", np.array(assetLabels)[shortIndex])

    #print("Optimal Weights: long positions: ", xStar[longIndex]) 
    #print("Optimal Weights: short positions: ", xStar[shortIndex])


    #print("Sum of long position weights: ",  np.sum(xStar[longIndex]))
    #print("Sum of short position weights:  ", np.sum(xStar[shortIndex]))
    #print("Sum of weights: " ,      np.sum(xStar))

    portfoliobeta = np.sum(np.multiply(xStar, betaAssets))
    #print("Portfolio Beta: ", portfoliobeta)


    print('Successful execution!')   
    
    salida = pd.DataFrame(list(zip(assetLabels,xStar))).transpose()
    salida.columns = salida.iloc[0]
    salida.drop([0], inplace=True)
    return salida

In [162]:
rfT = pd.read_csv('./Data/rf_test.csv')['three-month U.S. Treasury bill']
rfT[0]

0.0002

In [163]:
rfT

0     0.0002
1     0.0003
2     0.0001
3     0.0000
4     0.0022
5     0.0023
6     0.0028
7     0.0032
8     0.0053
9     0.0079
10    0.0106
11    0.0101
12    0.0144
13    0.0177
14    0.0198
15    0.0223
16    0.0240
17    0.0243
18    0.0221
19    0.0182
20    0.0155
21    0.0009
22    0.0009
Name: three-month U.S. Treasury bill, dtype: float64

In [172]:
portfolio_DE('./Data_De/2013-01-01-2014-12-31.csv', n_generations=700, rf=rfT[3])

generation: 1
HOF updated Generation 1
HOF fitness 7.1657627922049345
HOF updated Generation 1
HOF fitness 9.06875456615684
generation: 2
HOF updated Generation 2
HOF fitness 9.743787026334042
generation: 3
generation: 4
HOF updated Generation 4
HOF fitness 9.921634347134214
HOF updated Generation 4
HOF fitness 10.160193697626863
HOF updated Generation 4
HOF fitness 10.229560292107593
generation: 5
HOF updated Generation 5
HOF fitness 10.928174310595338
generation: 6
HOF updated Generation 6
HOF fitness 12.105671663714354
generation: 7
generation: 8
HOF updated Generation 8
HOF fitness 12.787835185963232
generation: 9
generation: 10
generation: 11
HOF updated Generation 11
HOF fitness 13.350521037667928
generation: 12
generation: 13
generation: 14
generation: 15
generation: 16
generation: 17
HOF updated Generation 17
HOF fitness 13.408189405662915
generation: 18
generation: 19
generation: 20
generation: 21
generation: 22
generation: 23
HOF updated Generation 23
HOF fitness 13.414620206

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION
1,0,0.0319638,0,0,0.0319638,0.0319638,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [165]:
input_dataDE = os.listdir('./Data_DE')

final_list = list()
for file in input_dataDE:
    if ('GSPC' not in file) and ('ipynb' not in file):
        final_list.append(file)

In [166]:
final_list

['2013-01-01-2014-12-31.csv',
 '2013-04-01-2015-03-31.csv',
 '2013-07-01-2015-06-30.csv',
 '2013-10-01-2015-09-30.csv',
 '2014-01-01-2015-12-31.csv',
 '2014-04-01-2016-03-31.csv',
 '2014-07-01-2016-06-30.csv',
 '2014-10-01-2016-09-30.csv',
 '2015-01-01-2016-12-31.csv',
 '2015-04-01-2017-03-31.csv',
 '2015-07-01-2017-06-30.csv',
 '2015-10-01-2017-09-30.csv',
 '2016-01-01-2017-12-31.csv',
 '2016-04-01-2018-03-31.csv',
 '2016-07-01-2018-06-30.csv',
 '2016-10-01-2018-09-30.csv',
 '2017-01-01-2018-12-31.csv',
 '2017-04-01-2019-03-31.csv',
 '2017-07-01-2019-06-30.csv',
 '2017-10-01-2019-09-30.csv',
 '2018-01-01-2019-12-31.csv',
 '2018-04-01-2020-03-31.csv',
 '2018-07-01-2020-06-30.csv']

In [173]:
salida = pd.DataFrame()
for i in range(23):
    begin = time.time() 
    path_aux = os.path.join('./Data_De/',final_list[i])
    temp_df = portfolio_DE(path_aux , n_generations=400, rf=rfT[i])
    end = time.time() 
    print(f"Total runtime of the program is {end - begin} seconds") 
    salida = pd.concat([salida, temp_df])
    print ("Iteracion:", i)

generation: 1
HOF updated Generation 1
HOF fitness 9.446069375537139
HOF updated Generation 1
HOF fitness 10.42816733996278
generation: 2
generation: 3
HOF updated Generation 3
HOF fitness 11.939106455603698
HOF updated Generation 3
HOF fitness 12.028203049197565
generation: 4
generation: 5
generation: 6
generation: 7
generation: 8
generation: 9
HOF updated Generation 9
HOF fitness 12.410822277891528
generation: 10
generation: 11
generation: 12
generation: 13
generation: 14
generation: 15
generation: 16
generation: 17
HOF updated Generation 17
HOF fitness 12.59527788803394
generation: 18
generation: 19
generation: 20
HOF updated Generation 20
HOF fitness 13.408996273839314
generation: 21
HOF updated Generation 21
HOF fitness 13.763808862277882
generation: 22
generation: 23
generation: 24
HOF updated Generation 24
HOF fitness 14.67672049660476
generation: 25
generation: 26
generation: 27
generation: 28
generation: 29
generation: 30
generation: 31
generation: 32
generation: 33
generation

In [174]:
salida

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADBE,ADI,...,XEL,XLNX,XOM,XRAY,XRX,XYL,YUM,ZBH,ZBRA,ZION
1,0,0.0345502,0,0,0.0345502,0,0,0,0.00755649,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0.0352738,0,0,0.0352738,0.0352738,0,0,0,0,...,0,0.00210353,0,0,0,0,0.0063035,0,0,0
1,0,0,0,0.0368543,0,0.0368543,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0368543,0
1,0,0,0,0,0,0.0317529,0,0,0,0,...,0,0,0,0,0,0,0,0,0.0317529,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0.0320315,0,0,0,0
1,0,0,0,0,0,0.0344667,0,0,0,0.0344667,...,0,0,0,0,0,0,0,0,0,0
1,0,0,-0.00465525,0,0,0.0331529,0,0,0.0331529,0,...,0,0,0,0,0,0,0,0,0,0
1,-0.000564861,0,0,0.0117683,0,0.0405679,-0.000564861,0,0.0405679,0,...,0,-0.000564861,0,0.000196115,0,0,0,0.000196115,0.000196115,0
1,0,0,0,0,0,0.0377597,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.000188579,0,0,0,0,0,0.000188579,0.000188579,0.000188579,0.000188579,...,0.039498,0,0,0.000188579,0.000188579,0.039498,0,0.000188579,0,0


In [175]:
salida.to_csv('./salida/portfolios_DE.csv' , index=False)